# Rule-Based Matching for RE

In [ ]:
# !pip install --upgrade spacy
!python -m spacy download en_core_web_lg
from pprint import pprint

2023-03-18 14:20:58.736326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 14:21:00.614870: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-18 14:21:00.615000: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-18 14:21:00.615024: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [ ]:
import spacy
from spacy.matcher import DependencyMatcher
from spacy import displacy

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
sentence = "United States works on legislation."
entities = ["United States", "legislation"]
relation = "work"

In [ ]:
doc = nlp(sentence)

In [ ]:
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
VERBS = ['ROOT', 'advcl']
OBJECTS = ["dobj", "dative", "attr", "oprd", 'pobj']

object_pronouns = ["me", "us", "them", "you", "him", "her", "it"]
subject_pronouns = ["i", "we", "they", "you", "he", "she", "it"]

subject_ls = []
object_ls = []

for token in doc:
  if token.dep_ in SUBJECTS and str(token.lower_) not in subject_pronouns:
    subject_ls.append(token.text)
  elif token.dep_ in OBJECTS and str(token.lower_) not in object_pronouns:
    object_ls.append(token.text)

print(subject_ls)
print(object_ls)

subjects = []
objects = []

for s in subject_ls:
  for e in entities:
    if s in e:
      subjects.append(e)

for o in object_ls:
  for e in entities:
    if o in e:
      objects.append(e)

print()
print(subjects)
print(objects)

['States']
['legislation']

['United States']
['legislation']


In [ ]:
def create_span_pattern(entity):
  doc = nlp(entity)
  if len(doc) == 1:
    return {"TEXT": doc[0].text}
  elif len(doc) == 2:
    return {"TEXT": doc[0].text, "TEXT": doc[1].text}
  elif len(doc) == 3:
    return {"TEXT": doc[0].text, "TEXT": doc[1].text, "TEXT": doc[2].text}
  elif len(doc) == 4:
    return {"TEXT": doc[0].text, "TEXT": doc[1].text, "TEXT": doc[2].text, "TEXT": doc[3].text}
  elif len(doc) == 5:
    return {"TEXT": doc[0].text, "TEXT": doc[1].text, "TEXT": doc[2].text, "TEXT": doc[3].text, "TEXT": doc[4].text}
  elif len(doc) == 6:
    return {"TEXT": doc[0].text, "TEXT": doc[1].text, "TEXT": doc[2].text, "TEXT": doc[3].text, "TEXT": doc[4].text, "TEXT": doc[5].text}
  elif len(doc) == 7:
    return {"TEXT": doc[0].text, "TEXT": doc[1].text, "TEXT": doc[2].text, "TEXT": doc[3].text, "TEXT": doc[4].text, "TEXT": doc[5].text, "TEXT": doc[6].text}
  elif len(doc) == 8:
    return {"TEXT": doc[0].text, "TEXT": doc[1].text, "TEXT": doc[2].text, "TEXT": doc[3].text, "TEXT": doc[4].text, "TEXT": doc[5].text, "TEXT": doc[6].text, "TEXT": doc[7].text}

In [ ]:
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
# create dependency and phrase matchers
dep_matcher = DependencyMatcher(nlp.vocab, validate=True)

pattern = [
    {
      "RIGHT_ID": "anchor_relation",
      "RIGHT_ATTRS": {"TEXT": "works"}
    },
    {
      "LEFT_ID": "anchor_relation",
      "REL_OP": ">>",
      "RIGHT_ID": "subject",
      "RIGHT_ATTRS": create_span_pattern("United States")
    },
    {
      "LEFT_ID": "anchor_relation",
      "REL_OP": ">>",
      "RIGHT_ID": "object",
      "RIGHT_ATTRS": create_span_pattern("legislation")
    }
  ]

# add pattern to extract SPO triples using the DEP matcher
dep_matcher.add("semantic_triple", [pattern])

In [ ]:
# extract matches of SPO triples
pso_matches = dep_matcher(doc)

pprint(pso_matches)

[(4699203773119030710, [2, 1, 4])]


---

In [ ]:
import json
import hashlib
import spacy
import os
from spacy.matcher import DependencyMatcher
from spacy.matcher import PhraseMatcher
#from models.statements import create_statement
# from pprint import pprint


def process_extract_triples():
  """###################################
  # Load/initialize main data structures and spaCy NLP pipeline
  ###################################"""

  SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
  VERBS = ['ROOT', 'advcl']
  OBJECTS = ["dobj", "dative", "attr", "oprd", 'pobj']

  object_pronouns = ["me", "us", "them", "you", "him", "her", "it"]
  subject_pronouns = ["i", "we", "they", "you", "he", "she", "it"]

  # load relations (a list)
  with open("shared_data/relations.json", encoding="utf8") as file:
    relations_dict = json.load(file)
  # load dictionary of relations/gazetteers (a list)
  with open("shared_data/relations_dictionary.json", encoding="utf8") as file:
    relations_dictionary = json.load(file)
  # load issues/entities (a list)
  with open("shared_data/issues_taxonomy.json", encoding="utf8") as file:
    entities_taxonomy = json.load(file)

  nlp = spacy.load("en_core_web_lg")

  rel_matcher = PhraseMatcher(nlp.vocab, None)

  # create pattern of phrase matcher for relations using gazetteers
  rel_patterns = [nlp.make_doc(gazetteer) for gazetteer in relations_dictionary]
  rel_matcher.add("relations", rel_patterns)


  def create_span_pattern(entity):
    _doc = nlp(entity)
    if len(_doc) == 1:
      return {"TEXT": _doc[0].text}
    elif len(_doc) == 2:
      return {"TEXT": _doc[0].text, "TEXT": _doc[1].text}
    elif len(_doc) == 3:
      return {"TEXT": _doc[0].text, "TEXT": _doc[1].text, "TEXT": _doc[2].text}
    elif len(_doc) == 4:
      return {"TEXT": _doc[0].text, "TEXT": _doc[1].text, "TEXT": _doc[2].text, "TEXT": _doc[3].text}
    elif len(_doc) == 5:
      return {"TEXT": _doc[0].text, "TEXT": _doc[1].text, "TEXT": _doc[2].text, "TEXT": _doc[3].text,
              "TEXT": _doc[4].text}
    elif len(_doc) == 6:
      return {"TEXT": _doc[0].text, "TEXT": _doc[1].text, "TEXT": _doc[2].text, "TEXT": _doc[3].text,
              "TEXT": _doc[4].text,
              "TEXT": _doc[5].text}
    elif len(_doc) == 7:
      return {"TEXT": _doc[0].text, "TEXT": _doc[1].text, "TEXT": _doc[2].text, "TEXT": _doc[3].text,
              "TEXT": _doc[4].text,
              "TEXT": _doc[5].text, "TEXT": _doc[6].text}
    elif len(_doc) == 8:
      return {"TEXT": _doc[0].text, "TEXT": _doc[1].text, "TEXT": _doc[2].text, "TEXT": _doc[3].text,
              "TEXT": _doc[4].text,
              "TEXT": _doc[5].text, "TEXT": _doc[6].text, "TEXT": _doc[7].text}

  # SPO triple function
  def extract_triples_from_sentence(_sent):

    # initial process of the sentence with the NLP pipeline
    doc = nlp(_sent["sentence"])

    #################################################
    # 1. use phrase matcher to identify occurrences of predicates and entities
    #################################################

    # matches for relation (1 or many extractions)
    rel_matches = rel_matcher(doc)
    # get relation spans from relation matches
    rel_spans = [doc[start:end] for _, start, end in rel_matches]
    # filter overlapping spans to ensure uniqueness of relations
    relation_spans = spacy.util.filter_spans(rel_spans)
    # create list of extracted relations in string type
    relations = [r.lemma_ for r in relation_spans]

    #################################################
    # 2. use relations (predicates) occurrence in sentence to identify entities
    #################################################

    # container of extracted triples. even though it may be unlikely,
    # a single sentence may contain more than 1 triple
    TRIPLES = []

    entities = _sent["entities"]
    # entities_list = [e[3] for e in entities]

    if relations and entities:
      # print(relations)

      subject_ls = []
      object_ls = []

      for token in doc:
        if token.dep_ in SUBJECTS and str(token.lower_) not in subject_pronouns:
          subject_ls.append([token.text, token.idx, token.idx + len(token.text) - 1])
        elif token.dep_ in OBJECTS and str(token.lower_) not in object_pronouns:
          object_ls.append([token.text, token.idx, token.idx + len(token.text) - 1])

      subjects = []
      objects = []

      for s in subject_ls:
        for e in entities:
          _range = range(e[0], e[1], 1)
          if s[1] in _range and s[2] in _range:
            subjects.append(e)

      for o in object_ls:
        for e in entities:
          _range = range(e[0], e[1], 1)
          if o[1] in _range and o[2] in _range:
            objects.append(e)

      if subjects and objects:

        pso_matches = []
        for r in relations:
          for s in subjects:
            for o in objects:
              # create dependency and phrase matchers
              dep_matcher = DependencyMatcher(nlp.vocab, validate=True)

              pattern = [
                {
                  "RIGHT_ID": "anchor_relation",
                  "RIGHT_ATTRS": {"LEMMA": r}
                },
                {
                  "LEFT_ID": "anchor_relation",
                  "REL_OP": ">",
                  "RIGHT_ID": "subject",
                  "RIGHT_ATTRS": create_span_pattern(s[3])
                },
                {
                  "LEFT_ID": "anchor_relation",
                  "REL_OP": ">",
                  "RIGHT_ID": "object",
                  "RIGHT_ATTRS": create_span_pattern(o[3])
                }
              ]

              # add pattern to extract SPO triples using the DEP matcher
              dep_matcher.add("semantic_triple", [pattern])
              # extract matches of SPO triples
              pso_matches1 = dep_matcher(doc)

              if pso_matches1:
                for m in pso_matches1:
                  pso_matches.append(m)

        if pso_matches:
          for pso_match in pso_matches:
            obj_token = doc[pso_match[1][2]]
            obj_match = [obj_token.idx, obj_token.idx + len(obj_token.text) - 1]
            # print(obj_match)
            rel_match = doc[pso_match[1][0]].text
            subj_token = doc[pso_match[1][1]]
            subj_match = [subj_token.idx, subj_token.idx + len(subj_token.text) - 1]

            _object = None
            _subject = None

            for o in objects:
              _range = range(o[0], o[1], 1)
              if obj_match[0] in _range and obj_match[1] in _range:
                _object = o[3]

            for s in subjects:
              _range = range(s[0], s[1], 1)
              if subj_match[0] in _range and subj_match[1] in _range:
                _subject = s[3]

            if _object and _subject and _object != _subject:
              obj_chunk = None
              subj_chunk = None
              for noun_chunk in doc.noun_chunks:

                if obj_token in noun_chunk and _object in noun_chunk.text:
                  if noun_chunk:
                    obj_chunk = noun_chunk.text
                  else:
                    obj_chunk = ""
                if subj_token in noun_chunk and _subject in noun_chunk.text:
                  if noun_chunk:
                    subj_chunk = noun_chunk.text
                  else:
                    subj_chunk = ""

              _triple = [
                (look_up_wikidata_id("entity", _subject), _subject, subj_chunk),
                (look_up_wikidata_id("relation", rel_match), rel_match),
                (look_up_wikidata_id("entity", _object), _object, obj_chunk)
              ]

              if _triple not in TRIPLES:
                TRIPLES.append(_triple)

    return TRIPLES